### 269. Alien Dictionary

**時間複雜度: $O( N + V + E )$**  
**空間複雜度: $O( V + E )$**

- N：所有字串長度的總和（即所有字母數量）
- V：所有不同的字母數量（graph 中的節點數）
- E：字母之間的順序關係（graph 中的邊數）

#### BFS

In [1]:
from typing import List
from collections import deque

class Solution:
    def foreignDictionary(self, words: List[str]) -> str:
        # 建立鄰接表（adjacency）表示圖中每個節點（字母）及其相鄰節點（後繼字母）
        adjacency = {} # space: O(V + E)
        for word in words: # time: O(N)
            for char in word:
                adjacency[char] = set()  # 先為每個出現的字母建立空集合（避免重複）

        # 建立入度表（indegree）記錄每個字母被多少其他字母指向
        indegree = {} # space: O(V)
        for char in adjacency: # time: O(V)
            indegree[char] = 0  # 初始入度皆為 0

        print(f"init: {adjacency=}")
        print(f"init: {indegree=}\n")

        # 逐對比較相鄰的單字，以找出字母間的先後順序關係 # time: O(N + E)
        for i in range(len(words) - 1): # time: O(N)
            word1 = words[i]
            word2 = words[i + 1]
            min_len = min(len(word1), len(word2))

            # 若 word1 是 word2 的前綴且比它長，代表無效（例如 ["abc", "ab"]）
            if (len(word1) > len(word2)) and (word1[:min_len] == word2[:min_len]):
                    return ""  # 無效字典，回傳空字串
                
            # 找出第一個不相同的字母，推斷出它們的順序
            for j in range(min_len):
                char1 = word1[j]
                char2 = word2[j]
                if char1 != char2:
                    # 若 char1 -> char2 尚未記錄，加入鄰接表並更新入度
                    if char2 not in adjacency[char1]:
                        adjacency[char1].add(char2) # time: O(1)，整體最多 O(E) 次邊的建立與入度更新操作
                        indegree[char2] += 1
                    break  # 找到第一個不同字母後即可停止內層比較

        print(f"update: {adjacency=}")
        print(f"update: {indegree=}\n")

        # 將所有入度為 0 的字母放入佇列（即沒有前置字母的節點）
        queue = deque([]) # space: O(V)
        for char, pre_count in indegree.items():
            if pre_count == 0:
                queue.append(char)
        
        print(f"{queue=}\n")

        result = [] # space: O(V)

        # 拓撲排序（BFS 寬度優先搜尋），time: O(V + E)
        while queue: # time: O(V)，每個節點（字母）會被放入與取出佇列各一次
            char = queue.popleft()
            result.append(char)

            # 將相鄰節點的入度減 1，若減為 0，表示可加入佇列
            for next_char in adjacency[char]: # time: O(E)，每條邊只會被走訪一次（減少入度）
                indegree[next_char] -= 1
                if indegree[next_char] == 0:
                    queue.append(next_char)

        print(f"init: {result=}")
        
        # 若結果字母數量不等於所有字母總數，代表存在循環（無法確定唯一順序）
        if len(result) != len(adjacency):
            return ""

        # 將拓撲排序結果轉為字串並回傳
        return "".join(result)


In [2]:
words = ["hrn","hrf","er","enn","rfnn"]
# Output: "hernf"

Solution().foreignDictionary(words)

init: adjacency={'h': set(), 'r': set(), 'n': set(), 'f': set(), 'e': set()}
init: indegree={'h': 0, 'r': 0, 'n': 0, 'f': 0, 'e': 0}

update: adjacency={'h': {'e'}, 'r': {'n'}, 'n': {'f'}, 'f': set(), 'e': {'r'}}
update: indegree={'h': 0, 'r': 1, 'n': 1, 'f': 1, 'e': 1}

queue=deque(['h'])

init: result=['h', 'e', 'r', 'n', 'f']


'hernf'

#### DFS

In [3]:
from typing import List

class Solution:
    def foreignDictionary(self, words: List[str]) -> str:
        # 建立鄰接表（adjacency）表示圖中每個節點（字母）及其相鄰節點（後繼字母）
        adjacency = {} # space: O(V + E)
        for word in words: # time: O(N)
            for char in word:
                adjacency[char] = set()  # 先為每個出現的字母建立空集合（避免重複）

        print(f"init: {adjacency=}")

        # 逐對比較相鄰的單字，以找出字母間的先後順序關係 # time: O(N + E)
        for i in range(len(words) - 1): # time: O(N)
            word1 = words[i]
            word2 = words[i + 1]
            min_len = min(len(word1), len(word2))

            # 若 word1 是 word2 的前綴且比它長，代表無效（例如 ["abc", "ab"]）
            if (len(word1) > len(word2)) and (word1[:min_len] == word2[:min_len]):
                    return ""  # 無效字典，回傳空字串
                
            # 找出第一個不相同的字母，推斷出它們的順序
            for j in range(min_len):
                char1 = word1[j]
                char2 = word2[j]
                if char1 != char2:
                    # 若 char1 -> char2 尚未記錄，加入鄰接表並更新入度
                    if char2 not in adjacency[char1]:
                        adjacency[char1].add(char2) # time: O(1)，整體最多 O(E) 次邊的建立與入度更新操作
                    break  # 找到第一個不同字母後即可停止內層比較

        print(f"update: {adjacency=}")


        # visited 用來記錄 DFS 狀態：
        #   - False: 正在遞迴訪問中 (Visiting / 檢測循環用)
        #   - True: 已完成訪問 (Processed / 該節點已加入結果)
        visited = {} # space: O(V)

        # 存放拓撲排序的結果（DFS 後序順序）
        result = [] # space: O(V)

        # 定義 DFS 遞迴函式
        def dfs(char):
            # 若節點已訪問過
            if char in visited:
                return visited[char]  # 若是 False 代表發現循環（非法），若 True 代表已完成

            visited[char] = False  # 標記為「正在訪問中」
            
            # 遞迴訪問所有相鄰節點
            for next_char in adjacency[char]: # time: O(E)
                state = dfs(next_char)
                if not state:  # 若發現循環，立即返回非法
                    return False
                
            visited[char] = True  # 訪問完成
            result.append(char)   # 將節點加入結果（後序加入）
            return True

        # 逐一對每個節點進行 DFS
        for char in adjacency:  # time: O(V + E)
            state = dfs(char)
            if not state:
                return ""  # 若發現循環，返回空字串
            
        result.reverse()  # 由於 DFS 產生的順序是反的，需要反轉

        return "".join(result)

In [4]:
words = ["hrn","hrf","er","enn","rfnn"]
# Output: "hernf"

Solution().foreignDictionary(words)

init: adjacency={'h': set(), 'r': set(), 'n': set(), 'f': set(), 'e': set()}
update: adjacency={'h': {'e'}, 'r': {'n'}, 'n': {'f'}, 'f': set(), 'e': {'r'}}


'hernf'